In [1]:
%reload_ext autoreload
%autoreload 3

import mne
import pandas as pd
import numpy as np
from src.preprocessing import utils
from pathlib import Path
import re

# data directories
data_path = f'/Volumes/Extreme SSD/PhD/MPI-LEMON/EEG_Raw_BIDS_ID/'
mpi_path = f'/Volumes/EEG_MPILMBB_LEMON/EEG_Preprocessed_BIDS_ID/EEG_Preprocessed/'

### Which data are excluded from MPI dataset for processing?

In [69]:
# get all subject ids in preprocessed folder
sub_ids_pro = []
for subject_path in sorted(Path(mpi_path).glob('sub-*')):
  sub_ids_pro.append(re.search('sub-(.*)_(.*)', subject_path.stem).group(1))
sub_ids_pro = set(sub_ids_pro)  # drop duplicates

# get all subject ids in raw folder
sub_ids_raw = []
for subject_path in sorted(Path(data_path).glob('sub-*')):
  sub_ids_raw.append(re.search('sub-(.*)', subject_path.stem).group(1))
sub_ids_raw = set(sub_ids_raw)  # drop any possible duplicates

# find any ids in sub_ids_raw that are not in sub_ids_pro
sub_ids_excluded = sorted(list(sub_ids_pro - sub_ids_raw))

## Annotations & Data Segmentation
### Data with different annotation patterns

In [124]:
fnames_annot_misbehave = {}
sub_ids_raw = []
for subject_path in sorted(Path(data_path).glob('sub-*')):
  sub_id = subject_path.stem
  sub_ids_raw.append(subject_path.stem)
  raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False)
  
  # check annot names
  points = utils.annotations_checker(raw.annotations.description.copy())

  # save those with deflection
  if sum(points.values()) != 2:
     fnames_annot_misbehave[sub_id] = points

### 1. segment and preprocess data with common annotation patterns

In [136]:
sub_ids_normal = sorted(list(set(sub_ids_raw) - set(fnames_annot_misbehave.keys())))
output_path = '/Volumes/Extreme SSD/PhD/MPI-LEMON/EEG_Raw_segmented/'
rsfreq = 250

In [138]:
for sub_id in sub_ids_normal[:5]:
    raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False)
    onsets, pattern = utils.find_pattern(raw)
    raws = utils.segment_raw(raw, onsets, pattern, duration=60)

    # save EC/EO segments in brainvision format
    ## create subject folder
    subject_path = output_path + sub_id
    Path(subject_path).mkdir(parents=True, exist_ok=True)
    ## downsample and save
    for k in raws.keys():
        raws[k].resample(rsfreq)
        raws[k].export(subject_path + f'/{sub_id}_{k}.vhdr')

### 2. segment raw data with not common annotation patterns

In [151]:
first_four = pd.DataFrame(fnames_annot_misbehave).T.query('all_same_type == True').index.values

In [163]:
pd.DataFrame(fnames_annot_misbehave).T.query('all_same_type == False').index.values

array(['sub-010126', 'sub-010138', 'sub-010155', 'sub-010157',
       'sub-010162', 'sub-010163', 'sub-010164', 'sub-010165',
       'sub-010166', 'sub-010168', 'sub-010228', 'sub-010233',
       'sub-010239', 'sub-010255', 'sub-010257', 'sub-010258',
       'sub-010260', 'sub-010261', 'sub-010262', 'sub-010263',
       'sub-010267', 'sub-010268', 'sub-010269', 'sub-010270',
       'sub-010271', 'sub-010272', 'sub-010273', 'sub-010274',
       'sub-010275', 'sub-010284', 'sub-010311', 'sub-010315',
       'sub-010318'], dtype=object)

In [ ]:
for sub_id in df[-7:]:
    raw = mne.io.read_raw_brainvision(data_path+f'/{sub_id}/RSEEG/{sub_id}.vhdr', verbose=False)
    print(f'>>>{sub_id}', utils.change_annot_names(raw.annotations.description.copy()), '\n')

In [ ]:
typ1 = ['sub-010026']
# an extera ''New Segment/', 'Comment/no USB Connection to actiCAP' markers in the beginning.

typ2 = ['sub-010030']
#an extera ''New Segment/', 'Comment/no USB Connection to actiCAP' markers in the beginning without extra switch markers.

typ3 = ['sub-010062', 'sub-010064', 'sub-010134'] # only one switch markers in the beginning 

typ4 = ['sub-010191'] # an extra 'New Segment/' 'Comment/no USB Connection to actiCAP' 'Stimulus/S  1' markers in the beginning.

typ5 = ['sub-010264'] # no switch markers in the beginning

# ----------------------------------------------
typ6 = ['sub-010126']
# this subject there is no switch markers, and the annotation for eye closed is 'Stimulus/S208'

# no switch markers in the annotations, and there is only two useless markers in the beginning
typ7 = ['sub-010138', 'sub-010155', 'sub-010157',
        'sub-010162', 'sub-010163', 'sub-010164', 'sub-010165',
        'sub-010166', 'sub-010168', 'sub-010228', 'sub-010233',
        'sub-010239', 'sub-010255', 'sub-010257', 'sub-010258',
        'sub-010260', 'sub-010261', 'sub-010262', 'sub-010263',
        'sub-010267', 'sub-010268', 'sub-010269', 'sub-010270',
        'sub-010271', 'sub-010272', 'sub-010273', 'sub-010274',
        'sub-010275', 'sub-010284', 'sub-010311', 'sub-010315',
        'sub-010318']